In [1]:
from datasets import load_dataset

In [3]:
dt = load_dataset('wikitext','wikitext-103-raw-v1')

HF google storage unreachable. Downloading and preparing it from source


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to C:/Users/Erfun/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
txt_dt_train = dt["train"]['text']

In [8]:
len(txt_dt_train)

1801350